In [16]:
import pickle
import pandas as pd
import numpy as np
from functions import get_predictors, save_artefact, load_artefact, set_result_predictions, set_btts_predictions, set_above_predictions, merge_all
from classes import Football_Model, MissingDict

In [75]:
map_values = {
        "Brighton and Hove Albion": "Brighton",
        "Leeds United": "Leeds",
        "Leiceister City": "Leiceister",
        "Manchester United": "Manchester Utd",
        "Newcastle United" : "Newcastle Utd",
        "Tottenham Hotspur": "Tot",
        "Tottenham" : "Tot",
        "Crystal Palace" : "CP",
        "West Ham United": "West Ham",
        "Wolverhampton Wanderers": "Wolves",
        "West Bromwich Albion" : "West Brom"
    }
mapping = MissingDict(**map_values)

In [6]:
matchweek = "Matchweek 19"
season = 2022
matches_to_predict_engineered = pd.read_csv("matches_to_predict_engineered.csv")
matches_to_predict = matches_to_predict_engineered[matches_to_predict_engineered["round"] == matchweek]
matches_to_predict = matches_to_predict[matches_to_predict["season"] == season]
matches_to_predict.head(1)

,Unnamed: 0.1,date,time,comp,round,day,venue,result,gf,ga,...,gca_rolling_3,tklw_rolling_3,int_rolling_3,tkl+int_rolling_3,err_rolling_3,succ_rolling_3,succ%_rolling_3,crdy_rolling_3,fls_rolling_3,won%_rolling_3
157,157,2023-01-03,19:45,Premier League,Matchweek 19,Tue,Home,NaN,NaN,NaN,...,4.666667,8.0,6.666667,21.333333,1.0,6.666667,40.833333,1.666667,10.0,40.333333


In [23]:
match_predictors, above_predictors, btts_predictors = get_predictors()

In [9]:
#Get all the tuned models and update the data for the different betting types
fm_target = load_artefact(f"{filepath}/target_tuned_model.pickle")
fm_target.update_data(matches_to_predict_engineered)

fm_btts = load_artefact(f"{filepath}/btts_tuned_model.pickle")
fm_btts.update_data(matches_to_predict_engineered)

fm_above = load_artefact(f"{filepath}/above_2.5_tuned_model.pickle")
fm_above.update_data(matches_to_predict_engineered)

### Getting Predictions for Random Forest

In [27]:
target_model_rf = fm_target.tuned_rf_model['model']
btts_model_rf = fm_btts.tuned_rf_model['model']
above_model_rf = fm_above.tuned_rf_model['model']

In [28]:
target_preds_rf = target_model_rf.predict(matches_to_predict[match_predictors])
btts_preds_rf = btts_model_rf.predict(matches_to_predict[btts_predictors])
above_preds_rf = above_model_rf.predict(matches_to_predict[above_predictors])

In [76]:
predicted_rf = matches_to_predict[['date','time','comp','team','opponent']]
predicted_rf['target_pred'] = target_preds_rf
predicted_rf['btts_pred'] = btts_preds_rf
predicted_rf['above_pred'] = above_preds_rf
predicted_rf['team'] = predicted_rf['team'].map(mapping)
predicted_rf['opponent'] = predicted_rf['opponent'].map(mapping)

C:\Users\Lee Seung Soo\AppData\Local\Temp\ipykernel_26712\2409860620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_rf['target_pred'] = target_preds_rf
C:\Users\Lee Seung Soo\AppData\Local\Temp\ipykernel_26712\2409860620.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_rf['btts_pred'] = btts_preds_rf
C:\Users\Lee Seung Soo\AppData\Local\Temp\ipykernel_26712\2409860620.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [77]:
predicted_rf

,date,time,comp,team,opponent,target_pred,btts_pred,above_pred
157,2023-01-03,19:45,Premier League,Arsenal,Newcastle Utd,1,0,1
243,2023-04-01,20:00,Premier League,Aston Villa,Wolves,0,0,1
326,2023-03-01,20:00,Premier League,Bournemouth,Manchester Utd,0,0,0
336,2023-01-02,17:30,Premier League,Brentford,Liverpool,1,0,1
483,2023-01-03,19:45,Premier League,Brighton,Everton,1,1,1
758,2023-05-01,20:00,Premier League,Chelsea,Manchester City,1,0,1
910,2023-04-01,20:00,Premier League,CP,Tot,0,0,1
1048,2023-01-03,19:45,Premier League,Everton,Brighton,0,0,0
1171,2023-04-01,19:45,Premier League,Leeds,West Ham,0,0,0
1324,2023-01-03,19:45,Premier League,Leicester City,Fulham,0,1,0


In [78]:
predicted_rf = predicted_rf.merge(predicted_rf, left_on = ["date", "team"], right_on = ["date", "opponent"])

In [79]:
predicted_rf

,date,time_x,comp_x,team_x,opponent_x,target_pred_x,btts_pred_x,above_pred_x,time_y,comp_y,team_y,opponent_y,target_pred_y,btts_pred_y,above_pred_y
0,2023-01-03,19:45,Premier League,Arsenal,Newcastle Utd,1,0,1,19:45,Premier League,Newcastle Utd,Arsenal,0,1,1
1,2023-04-01,20:00,Premier League,Aston Villa,Wolves,0,0,1,20:00,Premier League,Wolves,Aston Villa,0,1,0
2,2023-01-02,17:30,Premier League,Brentford,Liverpool,1,0,1,17:30,Premier League,Liverpool,Brentford,1,1,1
3,2023-01-03,19:45,Premier League,Brighton,Everton,1,1,1,19:45,Premier League,Everton,Brighton,0,0,0
4,2023-01-03,19:45,Premier League,Everton,Brighton,0,0,0,19:45,Premier League,Brighton,Everton,1,1,1
5,2023-04-01,19:45,Premier League,Leeds,West Ham,0,0,0,19:45,Premier League,West Ham,Leeds,0,1,1
6,2023-01-02,17:30,Premier League,Liverpool,Brentford,1,1,1,17:30,Premier League,Brentford,Liverpool,1,0,1
7,2023-01-03,19:45,Premier League,Newcastle Utd,Arsenal,0,1,1,19:45,Premier League,Arsenal,Newcastle Utd,1,0,1
8,2023-04-01,19:45,Premier League,West Ham,Leeds,0,1,1,19:45,Premier League,Leeds,West Ham,0,0,0
9,2023-04-01,20:00,Premier League,Wolves,Aston Villa,0,1,0,20:00,Premier League,Aston Villa,Wolves,0,0,1


In [ ]:
predicted_rf.to_csv('FINAL RF PREDICTION.csv')